In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
import pandas as pd

# ---- timeslot 1 ----
# df_1   = pd.read_csv('20250624_timeslot_1_cleaned.csv').drop(
#     columns=['gps_X', 'gps_Y', 'Time', 'Filename', 'Location',
#              'temp', 'rh', 'gps_ele', 'gps_hdop'], errors='ignore'
# )
# df_1_d = pd.read_csv('20250624_timeslot_1_derivative_data_cleaned.csv').drop(
#     columns=['temp', 'rh', 'gps_ele', 'gps_hdop', 'gps_X', 'gps_Y', 'Location'], errors='ignore'
# )

# # ---- timeslot 2 ----
# df_2   = pd.read_csv('20250624_timeslot_2_cleaned.csv').drop(
#     columns=['gps_X', 'gps_Y', 'Time', 'Filename', 'Location',
#              'temp', 'rh', 'gps_ele', 'gps_hdop'], errors='ignore'
# )
# df_2_d = pd.read_csv('20250624_timeslot_2_derivative_data_cleaned.csv').drop(
#     columns=['temp', 'rh', 'gps_ele', 'gps_hdop', 'gps_X', 'gps_Y', 'Location'], errors='ignore'
# )

# # ---- timeslot 3 ----
# df_3   = pd.read_csv('20250602_timeslot_3_cleaned.csv').drop(
#     columns=['gps_X', 'gps_Y', 'Time', 'Filename', 'Location',
#              'temp', 'rh', 'gps_ele', 'gps_hdop'], errors='ignore'
# )
# df_3_d = pd.read_csv('20250602_timeslot_3_derivative_data.csv').drop(
#     columns=['temp', 'rh', 'gps_ele', 'gps_hdop', 'gps_X', 'gps_Y', 'Location'], errors='ignore'
# )

# ---- all timeslots ----
df_all   = pd.read_csv('all_timeslot_cleaned.csv').drop(
    columns=['gps_X', 'gps_Y', 'Time', 'Filename', 'Location',
             'temp', 'rh', 'gps_ele', 'gps_hdop', 'month'], errors='ignore'
)
df_all_d = pd.read_csv('all_timeslot_derivative_data_cleaned.csv').drop(
    columns=['temp', 'rh', 'gps_ele', 'gps_hdop', 'gps_X', 'gps_Y', 'Location'], errors='ignore'
)


In [ ]:
df_all

In [ ]:
df_all_d

In [ ]:
## X is the actual data

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

def scale_dataframes(df, columns_start):
    X = df.iloc[:, columns_start:]
    Y = df.iloc[:, :columns_start]
    
    scaler_X = StandardScaler()
    X_scaled = pd.DataFrame(scaler_X.fit_transform(X), columns=X.columns)

    scaler_Y = StandardScaler()
    Y_scaled = pd.DataFrame(scaler_Y.fit_transform(Y), columns=Y.columns)
    
    return X_scaled, Y_scaled

# X_1 = df_1.iloc[:, 7:]
# X_2 = df_2.iloc[:, 7:]
# X_3 = df_3.iloc[:, 7:]
X_all = df_all.iloc[:, 7:]

# Y_1 = df_1.iloc[:, :7]
# Y_2 = df_2.iloc[:, :7]
# Y_3 = df_3.iloc[:, :7]
Y_all = df_all.iloc[:, :7]

# X_1_d = df_1_d.iloc[:, 7:]
# X_2_d = df_2_d.iloc[:, 7:]
# X_3_d = df_3_d.iloc[:, 7:]
X_all_d = df_all_d.iloc[:, 7:]

# Y_1_d = df_1_d.iloc[:, :7]
# Y_2_d = df_2_d.iloc[:, :7]
# Y_3_d = df_3_d.iloc[:, :7]
Y_all_d = df_all_d.iloc[:, :7]

# Scale original dataframes
# X_1_scaled, Y_1_scaled = scale_dataframes(df_1, 7)
# X_2_scaled, Y_2_scaled = scale_dataframes(df_2, 7)
# X_3_scaled, Y_3_scaled = scale_dataframes(df_3, 7)
X_all_scaled, Y_all_scaled = scale_dataframes(df_all, 7)

# Scale additional dataframes
# X_1_d_scaled, Y_1_d_scaled = scale_dataframes(df_1_d, 7)
# X_2_d_scaled, Y_2_d_scaled = scale_dataframes(df_2_d, 7)
# X_3_d_scaled, Y_3_d_scaled = scale_dataframes(df_3_d, 7)
X_all_d_scaled, Y_all_d_scaled = scale_dataframes(df_all_d, 7)

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# # # Set the size of the figure
# plt.figure(figsize=(15, 10))

# # Create the heatmap with enhancements
# sns.heatmap(X_2.corr(), annot=True, fmt=".2f", cmap='coolwarm', 
#             linewidths=0.5,)

# # Add a title with enhanced formatting
# plt.title('Correlation Heatmap', fontweight='bold')

# # Show the plot
# plt.show()

## 用XGBoost测试关系

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (
    r2_score, mean_absolute_error, mean_squared_error,
    explained_variance_score, median_absolute_error
)
import shap
from typing import List
from tqdm import tqdm

def evaluate_xgboost(X: pd.DataFrame, y: pd.Series, test_size=0.1, cv_folds=5, name='VOC', shap_top_k=10, plot_force_example=False):
    """
    Train XGBRegressor, report metrics, make diagnostic plots, and add SHAP analysis.
    """
    # Split into train/test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42, shuffle=True
    )
    print(f'Training size: {len(X_train)}, Testing size: {len(X_test)}')

    # Instantiate and fit the model
    model = XGBRegressor(objective='reg:squarederror', random_state=42)
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Calculate metrics
    metrics = {
        'r2': r2_score(y_test, y_pred),
        'mae': mean_absolute_error(y_test, y_pred),
        'mse': mean_squared_error(y_test, y_pred),
        'rmse': np.sqrt(mean_squared_error(y_test, y_pred)),
        'explained_var': explained_variance_score(y_test, y_pred),
        'median_ae': median_absolute_error(y_test, y_pred)
    }

    # Baseline model
    y_baseline = np.mean(y_train)
    r2_baseline = r2_score(y_test, np.full(y_test.shape, y_baseline))

    # Print results
    print(f"R² for Baseline Model: {r2_baseline:.4f}")
    print(f"R² for XGBoost Model: {metrics['r2']:.4f}")

    # Visualization
    sns.set(style="whitegrid")
        # Visualization
    sns.set(style="whitegrid")
    fig, axs = plt.subplots(2, 2, figsize=(18, 14))

    # 1. Scatter plot: True vs Predicted
    axs[0, 0].scatter(y_test, y_pred, alpha=0.6, color="#0072B2", edgecolor='k', s=60)
    axs[0, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=3)
    axs[0, 0].set_xlabel("True Values", fontsize=14)
    axs[0, 0].set_ylabel("Predicted Values", fontsize=14)
    axs[0, 0].set_title(f"True vs Predicted Values ({name})", fontsize=16, fontweight='bold')
    axs[0, 0].grid(True, which='both', linestyle='--', linewidth=0.5)
    axs[0, 0].tick_params(axis='both', labelsize=12)
    min_val = min(y_test.min(), y_pred.min())
    max_val = max(y_test.max(), y_pred.max())
    axs[0, 0].set_xlim(min_val, max_val)
    axs[0, 0].set_ylim(min_val, max_val)
    axs[0, 0].set_aspect('equal', adjustable='box')

    # 2. Prediction error plot (regression line)
    sns.regplot(
        x=y_test, y=y_pred, ax=axs[0, 1],
        scatter_kws={"alpha": 0.6, "color": "#D55E00", "edgecolor": "k", "s": 60},
        line_kws={"color": "navy", "lw": 3}
    )
    axs[0, 1].set_xlabel("True Values", fontsize=14)
    axs[0, 1].set_ylabel("Predicted Values", fontsize=14)
    axs[0, 1].set_title(f"Prediction Error Plot ({name})", fontsize=16, fontweight='bold')
    axs[0, 1].grid(True, linestyle='--', linewidth=0.5)
    axs[0, 1].tick_params(axis='both', labelsize=12)

    # 3. Feature importances bar chart
    coefficients = model.feature_importances_
    features = X.columns
    indices = np.argsort(coefficients)[::-1]
    colors = sns.color_palette("coolwarm", len(coefficients))
    axs[1, 0].barh(range(len(coefficients)), coefficients[indices][::-1],
                   color=colors[::-1], edgecolor='black')
    axs[1, 0].set_yticks(range(len(coefficients)))
    axs[1, 0].set_yticklabels(features[indices][::-1], fontsize=12)
    axs[1, 0].invert_yaxis()
    axs[1, 0].set_xlabel("Importance Value", fontsize=14)
    axs[1, 0].set_title(f"Coefficients ({name})", fontsize=16, fontweight='bold')
    axs[1, 0].grid(axis='x', linestyle='--', linewidth=0.5)

    # 4. Distribution of Actual vs Predicted
    sns.kdeplot(y_test, label=f'Actual {name}', fill=True, ax=axs[1, 1], color="#1B9E77")
    sns.kdeplot(y_pred, label=f'Predicted {name}', fill=True, ax=axs[1, 1], color="#D95F02")
    axs[1, 1].set_title(f'Distribution of Actual vs Predicted ({name})', fontsize=16, fontweight='bold')
    axs[1, 1].set_xlabel(f'{name}', fontsize=14)
    axs[1, 1].legend(fontsize=12)
    axs[1, 1].tick_params(axis='both', labelsize=12)
    axs[1, 1].grid(True, linestyle='--', linewidth=0.5)

    plt.tight_layout()
    plt.show()

    # =====================
    # SHAP ANALYSIS SECTION
    # =====================
    print("\nComputing SHAP values...")
    try:
        explainer = shap.TreeExplainer(model)
        shap_values = explainer(X_test)
    except Exception as e:
        print(f"TreeExplainer failed: {e}. Falling back to shap.Explainer.")
        explainer = shap.Explainer(model, X_train)
        shap_values = explainer(X_test)
    
    # Summary plots
    plt.figure(figsize=(10, 6))
    shap.summary_plot(shap_values, X_test, show=False)
    
    # Customize title and labels
    plt.title(f"SHAP Summary - {name}", fontsize=20, fontweight='bold')
    # plt.xlabel("SHAP Value", fontsize=20)
    # plt.ylabel("Features", fontsize=20)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=20)
    
    plt.tight_layout()
    plt.show()
    
    plt.figure(figsize=(10, 6))
    shap.summary_plot(shap_values, X_test, plot_type="bar", show=False)
    
    # Customize title and labels
    plt.title(f"SHAP Summary (Bar) - {name}", fontsize=20, fontweight='bold')
    # plt.xlabel("Mean Absolute SHAP Value", fontsize=20)
    # plt.ylabel("Features", fontsize=20)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=20)
    
    plt.tight_layout()
    plt.show()

        
    return metrics

In [ ]:
from typing import List
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

def results_summary(
    air_type: str, 
    X_in: List[pd.DataFrame], 
    Y_in: List[pd.DataFrame], 
    X_in_names: List[str], 
    Y_in_names: List[str]
) -> None:
    """
    Generate a results summary matrix and heatmap for given X and Y inputs.
    
    Args:
        air_type (str): The target air quality type (e.g., 'VOC').
        X_in (list): List of feature input DataFrames.
        Y_in (list): List of target input DataFrames.
        X_in_names (list): Names corresponding to X_in.
        Y_in_names (list): Names corresponding to Y_in.
    """
    # Initialize an empty DataFrame for results
    results_matrix = pd.DataFrame(index=X_in_names, columns=Y_in_names)
    
    # Fill the matrix with evaluation results
    for i, x in enumerate(tqdm(X_in, desc="X Loop")):
        for j, y in enumerate(tqdm(Y_in, desc="Y Loop")):
            try:
                print(f"\n → X: {X_in_names[i]}, Y: {Y_in_names[j]}")
                metrics = evaluate_xgboost(x, y[air_type], name=air_type)
                results_matrix.iloc[i, j] = round(metrics['r2'], 4) 
            except ValueError as e:
                # Handle inconsistent sample size or other ValueErrors
                print(f"Skipping pair (X: {X_in_names[i]}, Y: {Y_in_names[j]}) due to error: {e}")
                results_matrix.iloc[i, j] = None  # Mark as empty or 'None'

    # Display the results matrix
    print("\nResults Summary Matrix:")
    print(results_matrix)

    # Plot a heatmap, skipping NaN values
    plt.figure(figsize=(4, 3))
    sns.heatmap(
        results_matrix.astype(float), 
        annot=True, 
        cmap='coolwarm', 
        xticklabels=Y_in_names, 
        yticklabels=X_in_names, 
        cbar_kws={'label': 'R² Value'},
        mask=results_matrix.isnull()  # Mask NaN values
    )
    plt.title(f'R² Values Heatmap for {air_type}')
    plt.xlabel('Y Inputs')
    plt.ylabel('X Inputs')
    plt.show()

# # Define placeholders for potentially missing variables
# X_3 = X_3_scaled = X_3_d = X_3_d_scaled = None
# Y_3 = Y_3_scaled = Y_3_d = Y_3_d_scaled = None

def choose_timeslot(
    timeslot: str = '1', 
    air_type: str = 'VOC'
) -> None:
    """
    Choose the appropriate timeslot and generate results summary.

    Args:
        timeslot (str): Timeslot to use ('1', '2', '3', or 'all').
        air_type (str): The target air quality type (e.g., 'VOC').
    """
    # Define the timeslot mapping
    timeslot_mapping = {
        # '1': {
        #     'X_input': [X_1, X_1_scaled, X_1_d, X_1_d_scaled],
        #     'Y_input': [Y_1, Y_1_scaled, Y_1_d, Y_1_d_scaled],
        #     'X_names': ['X_1', 'X_1_scaled', 'X_1_d', 'X_1_d_scaled'],
        #     'Y_names': ['Y_1', 'Y_1_scaled', 'Y_1_d', 'Y_1_d_scaled']
        # },
        # '2': {
        #     'X_input': [X_2, X_2_scaled, X_2_d, X_2_d_scaled],
        #     'Y_input': [Y_2, Y_2_scaled, Y_2_d, Y_2_d_scaled],
        #     'X_names': ['X_2', 'X_2_scaled', 'X_2_d', 'X_2_d_scaled'],
        #     'Y_names': ['Y_2', 'Y_2_scaled', 'Y_2_d', 'Y_2_d_scaled']
        # },
        # '3': {
        #     'X_input': [X_3, X_3_scaled, X_3_d, X_3_d_scaled],
        #     'Y_input': [Y_3, Y_3_scaled, Y_3_d, Y_3_d_scaled],
        #     'X_names': ['X_3', 'X_3_scaled', 'X_3_d', 'X_3_d_scaled'],
        #     'Y_names': ['Y_3', 'Y_3_scaled', 'Y_3_d', 'Y_3_d_scaled']
        # },
        'all': {
            'X_input': [X_all, X_all_scaled, X_all_d, X_all_d_scaled],
            'Y_input': [Y_all, Y_all_scaled, Y_all_d, Y_all_d_scaled],
            'X_names': ['X_all', 'X_all_scaled', 'X_all_d', 'X_all_d_scaled'],
            'Y_names': ['Y_all', 'Y_all_scaled', 'Y_all_d', 'Y_all_d_scaled']
        }
    }
    
    # Ensure the timeslot is valid
    if timeslot not in timeslot_mapping:
        raise ValueError(f"Invalid timeslot '{timeslot}'. Choose from '1', '2', '3', or 'all'.")

    # Get inputs and names for the selected timeslot
    selected_timeslot = timeslot_mapping[timeslot]
    X_input = selected_timeslot['X_input']
    Y_input = selected_timeslot['Y_input']
    X_names = selected_timeslot['X_names']
    Y_names = selected_timeslot['Y_names']

    # Generate the results summary
    results_summary(air_type=air_type, X_in=X_input, Y_in=Y_input, X_in_names=X_names, Y_in_names=Y_names)

## Time slot 1

In [ ]:
# choose_timeslot('1','VOC')

In [ ]:
# choose_timeslot('1','CO')

In [ ]:
# choose_timeslot('1','NO2')

In [ ]:
# choose_timeslot('1','SO2')

In [ ]:
# choose_timeslot('1','PM25')

In [ ]:
# choose_timeslot('1','PM10')

In [ ]:
# choose_timeslot('1','O3')

## Time slot 2

In [ ]:
# choose_timeslot('2','VOC')

In [ ]:
# choose_timeslot('2','CO')

In [ ]:
# choose_timeslot('2','NO2')

In [ ]:
# choose_timeslot('2','SO2')

In [ ]:
# choose_timeslot('2','PM25')

In [ ]:
# choose_timeslot('2','PM10')

In [ ]:
# choose_timeslot('2','O3')

## Time slot 3

In [ ]:
# choose_timeslot('3','VOC')

In [ ]:
# choose_timeslot('3','CO')

In [ ]:
# choose_timeslot('3','NO2')

In [ ]:
# choose_timeslot('3','SO2')

In [ ]:
# choose_timeslot('3','PM25')

In [ ]:
# choose_timeslot('3','PM10')

In [ ]:
# choose_timeslot('3','O3')

## Timeslot all

In [ ]:
choose_timeslot('all','VOC')

In [ ]:
choose_timeslot('all','CO')

In [ ]:
choose_timeslot('all','NO2')

In [ ]:
choose_timeslot('all','SO2')

In [ ]:
choose_timeslot('all','PM25')

In [ ]:
choose_timeslot('all','PM10')

In [ ]:
choose_timeslot('all','O3')